# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [58]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [60]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [31]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [16]:
print(get_links_user_prompt("https://edwarddonner.com"))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
http

In [ ]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [33]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [20]:
select_relevant_links("https://edwarddonner.com")

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-Realtime-0.5B',
 '/Tongyi-MAI/Z-Image-Turbo',
 '/zai-org/GLM-4.6V-Flash',
 '/zai-org/GLM-4.6V',
 '/mistralai/Devstral-Small-2-24B-Instruct-2512',
 '/models',
 '/spaces/Tongyi-MAI/Z-Image-Turbo',
 '/spaces/mrfakename/Z-Image-Turbo',
 '/spaces/OpenEvals/evaluation-guidebook',
 '/spaces/dream2589632147/Dream-wan2-2-faster-Pro',
 '/spaces/anycoderapps/VibeVoice-Realtime-0.5B',
 '/spaces',
 '/datasets/Anthropic/AnthropicInterviewer',
 '/datasets/TuringEnterprises/Turing-Open-Reasoning',
 '/datasets/nvidia/PhysicalAI-Autonomous-Vehicles',
 '/datasets/nvidia/ToolScale',
 '/datasets/openai/gdpval',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon'

In [23]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Zhihu page', 'url': 'https://www.zhihu.com/org/huggingface'},
  {'type': 'Community forum', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [33]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Found links: {'links': [{'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Product endpoints', 'url': 'https://endpoints.huggingface.co'}, {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'}, {'type'

In [50]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [25]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [57]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n3 days ago\n•\n80.2k\n•\n733\nTongyi-MAI/Z-Image-Turbo\nUpdated\n3 days ago\n•\n245k\n•\n2.55k\nzai-org/GLM-4.6V-Flash\nUpdated\n2 days ago\n•\n18.6k\n•\n350\nzai-org/GLM-4.6V\nUpdated\n3 days ago\n•\n2.21k\n•\n268\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\n1 day ago\n•\n7.22k\n•\n267\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMC

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: The AI community building the future

The AI community building the future. The platform where the machine learning community collaborates on models, datasets, and applications.

---

## Who we are (in a sentence and a joke)

- The Home of Machine Learning, where creators, researchers, and everyone in between can create, discover, and collaborate on ML faster.
- A global open-source playground with enterprise-grade options for teams that need a little more gravity than a weekend hackathon.
- If your model could use a bigger crowd, you’re in the right place.

Humor aside: we’re basically the coffee shop for ML—lots of activity, great people, and everyone leaves with a better model.

---

## What you can do with Hugging Face

- Browse 1M+ models and explore AI Apps
- Run Spaces and deploy your ideas with a click
- Use an open-source stack that moves the field forward
- Access 45,000+ models from leading AI providers through a single, unified API with no service fees
- Work with diverse modalities: text, image, video, audio, and even 3D

Open-source backbone (the brain of the operation):
- Transformers, Diffusers, Tokenizers, Safetensors, PEFT, TRL, Transformers.js, and more
- Datasets, Text Generation Inference, Accelerate, and other tools that help you build faster

---

## Enterprise & pricing (for teams that mean business)

- Team & Enterprise plans starting at $20 per user per month
- Single Sign-On, Regions, Priority Support, Audit Logs, Resource Groups, Private Datasets Viewer
- Inference Endpoints: deploy on optimized endpoints or scale your Spaces with GPU in a few clicks
- GPU pricing starting at $0.60/hour
- More than 50,000 organizations are using Hugging Face (yep, a lot of friends and colleagues)

Ready to go from hobbyist corner to enterprise-grade? We’ve got you.

---

## Customers & community

- Trusted by a global roster of organizations and teams
- Examples include AI2 Team, AI at Meta, Amazon, Google, Intel, Microsoft, Grammarly, Writer
- It’s not just about the tech; it’s about the people building, sharing, and improving together

The platform is built for collaboration: host and collaborate on unlimited public models, datasets, and applications, and accelerate your ML work with the Open Source stack.

---

## Careers & culture

- We’re building the foundation of ML tooling with the community.
- A culture of collaboration, openness, and curiosity—plus a dash of humor to keep things human.
- See our Careers page for the latest openings and opportunities to join the team that’s shaping the future of AI.

If you like working with smart people on impactful projects, you’ll feel right at home here.

---

## Get started quickly

- Sign up and explore: Browse 1M+ models, explore AI Apps, or dive into Spaces
- Sign in to access tools that fit your workflow
- Start building with a robust open-source stack and enterprise-ready options

Bonus: you’ll be in excellent company—your models will thank you, your data will thank you, and your ego might need a little room to swell.

---

## Quick pull quotes (so you can skim)

- “The AI community building the future.” 
- “The platform where the machine learning community collaborates on models, datasets, and applications.”
- “Move faster with the HF Open source stack.”
- “Access 45,000+ models from leading AI providers through a single, unified API with no service fees.”
- “More than 50,000 organizations are using Hugging Face.”

---

Ready to join the party? Explore Models, Datasets, Spaces, and more at Hugging Face today.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [54]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [48]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face – The AI community building the future

The platform where the machine learning community collaborates on models, datasets, and applications.

---

## Why Hugging Face

- Open, community-driven foundation for ML tooling and collaboration
- Host and collaborate on unlimited public models, datasets, and applications
- Move faster with the HF Open Source stack
- Support for all modalities: text, image, video, audio, or even 3D
- Build your ML portfolio and share your work with the world

---

## Products & Platforms

- Models
  - Browse 1M+ models from the community and leading providers
  - Access 45,000+ models from various providers through a single, unified API
- Datasets
  - Access and share datasets for any ML task
- Spaces
  - Run interactive ML apps, demos, and experiments
- Enterprise & Pricing
  - Team & Enterprise plans with enterprise-grade security, access controls, and dedicated support
  - Getting started from $20 per user/month (significant features like SSO, audit logs, regions, etc.)
  - Compute options: Inference Endpoints and GPU-backed Spaces
- Inference & Acceleration
  - Deploy on optimized Inference Endpoints
  - GPU-based scaling options starting at $0.60/hour
- Community & Docs
  - Rich documentation, forums, and a thriving developer community
- Sign Up / Sign In
  - Join the community to start creating, sharing, and collaborating

---

## Our Open Source Foundation

We’re building the foundation of ML tooling with a vibrant open source ecosystem. Notable projects and their scale include:

- Transformers: 153,816
- Diffusers: 32,040
- Safetensors: 3,548
- Hub Python Library: 3,131
- Tokenizers: 10,302
- TRL: 16,635
- Transformers.js: 15,080
- smolagents: 24,406
- PEFT: 20,269

Plus, a broad ecosystem around Datasets, Text Generation Inference, and Accelerate.

---

## Customers & Partnerships

Join 50,000+ organizations leveraging Hugging Face for ML development, deployment, and collaboration. Notable customers and partners include:

- Ai2 (non-profit)
- AI at Meta
- Amazon
- Google
- Intel
- Microsoft
- Grammarly
- Writer

These organizations rely on Hugging Face for access to vast model catalogs, datasets, and scalable deployment.

---

## Careers & Culture

- Hugging Face publishes a Careers page with opportunities across teams.
- Culture emphasizes collaboration, openness, and contribution to the open source ML ecosystem.
- A community-first mindset: collaborate on models, datasets, and apps to advance the field together.
- If you’re passionate about AI, open source tooling, and building with others, consider joining the team.

---

## Get Involved

- Explore Models, Datasets, and Spaces
- Sign up to build your ML portfolio and share your work
- For enterprises, explore Team & Enterprise plans with security, governance, and dedicated support
- Visit the Careers page to learn about current openings

Links you might find useful:
- Docs, Blog, Forum
- Inference Endpoints, Spaces, and Models
- Sign Up / Log In
- Careers, Press, About, Privacy, Terms
- Social: GitHub, Twitter, LinkedIn, Discord

---

Hugging Face is the AI community building the future — a collaborative, open, and scalable platform for models, data, and applications. Whether you’re a researcher, engineer, data journalist, product builder, or a prospective recruit, there’s a place for you in the community.

In [56]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face — The AI community building the future

Welcome to the place where models, datasets, and spaces make friends, not friction. We’re the collaboration platform for the machine learning crowd — from researchers to developers to folks who just want to see cool AI in action.

## Who we are (in one sentence, with snacks)

- The Home of Machine Learning: a supercharged hub to create, discover, and collaborate on ML projects.
- The Open Source Engine: our stack powers transformers, diffusers, and a bazillion tiny miracles that make ML go vroom.
- The Community Engine: a global crew of engineers, scientists, and end users learning, sharing, and cheering each other on.

Humor aside, we mean business: innovation, openness, and helping you move faster than a caffeinated dataset.

## What you can do here

- Browse 1M+ models: from zippy image generators to speech founts and everything in between.
- Explore 400k+ applications: deploy ideas as live apps you can actually use.
- Dive into 250k+ datasets: practice, test, and train with real-world data.
- Run Spaces: host and run your ML apps with a few clicks (they’re “Running” and sometimes “MCP” — we’re keeping you on your toes).

For the cool kids who like to go beyond plain text:
- Work across all modalities: text, image, video, audio, or even 3D.

## Our open source stack (the backbone of modern ML)

“Open source first” isn’t a slogan here — it’s how we breathe.
- Transformers, Diffusers, Tokenizers, and more — with millions of models and tools at your fingertips.
- Safe, easy storage and transfer with Safetensors.
- In-browser ML with Transformers.js.
- Tools to fine-tune efficiently (PEFT) and build smarter agents (smolagents).

If you like APIs, you’ll love the single unified API that reaches 45,000+ models from leading AI providers — with no service fees. We’re basically the Swiss Army knife for ML developers.

## Why customers love Hugging Face (and why investors might grin)

- A massive, growing community of users and contributors (50,000+ organizations rely on us).
- Enterprise-grade options: Team & Enterprise features include security, access controls, priority support, SSO, audit logs, resource grouping, private datasets, and more.
- Compute that scales: deploy on optimized Inference Endpoints or upgrade Spaces to GPU in just a few clicks.
- Transparent pricing that starts at approachable levels (Getting started: from $20/user/month; GPU compute from $0.60/hour).
- A portfolio of big-name users and partners (Meta, Amazon, Google, Intel, Microsoft, Grammarly, Writer, and many more) showing that real organizations trust our platform.

And yes, we’re serious about safety, governance, and making ML tools accessible to builders of all stripes. Hugging Face isn’t just a brand — it’s a movement toward collaboration, openness, and shared progress.

## Culture you’ll actually want to work with

- Open, collaborative, and community-driven: your ideas matter; your pull requests matter more.
- Cross-disciplinary vibes: engineers, researchers, data folks, designers, and product folks all building together.
- A love for sharing what we learn: docs, blogs, forums, and discussions that help everyone level up.
- A sense of humor about AI, and a passion for making tools that are powerful yet approachable.

We’re not just about doing cool stuff; we’re about doing it together and helping the next generation of ML engineers, scientists, and end users learn, collaborate, and share their work with the world.

## Careers & opportunities

- Careers at Hugging Face: a place where you can build, contribute, and grow in a culture of openness and collaboration.
- What you’ll find: a team that values initiative, curiosity, and a friendly poke at each other’s code (in good spirits).
- Roles span engineering, research, product, operations, and community/community governance — all with a front-row seat to the future of ML tooling.
- Perks include meaningful work on a thriving open-source stack, opportunities to impact ML at scale, and a place in a global community of ML enthusiasts.

If you’re excited by the idea of shaping the next era of AI tooling, you’ll feel right at home.

## Brand vibe (so you know what we look like)

- Colors: vibrant HF yellow/orange hues (#FFD21E, #FF9D00) with the calm neutral gray (#6B7280).
- Brand promise: the collaboration platform for the machine learning community.
- Visuals emphasize sharing, discovery, and experimentation in an open, friendly way.

Brand assets and a clear identity back up a culture that’s playful, pragmatic, and built for collaboration.

## Quick-start for you

- Sign Up and join the community.
- Explore 1M+ models, 400k+ apps, and 250k+ datasets.
- Use Spaces to run your apps or spin up GPUs in a few clicks.
- If you’re a team or enterprise, check out Team & Enterprise for security, governance, and dedicated support.
- Dive into our open-source stack to see what powers modern ML tooling.

## Closing note (because who doesn’t like a good send-off?)

Hugging Face is more than a platform — it’s a friendly, ambitious ecosystem where you can hug (figuratively, of course) the future of AI by collaborating with the world. Whether you’re a customer shipping production ML, an investor watching the network effects unfold, or a candidate hunting for a meaningful, fun career, we’ve got a seat at the table.

Want to learn more or start building with us? Sign up, explore, and let’s Hugging Face the future together.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>